In [ ]:
# 🎮 安装所需依赖
!pip install torch deepctr-torch pandas numpy scikit-learn flask requests pyngrok matplotlib

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple

[notice] A new release of pip is available: 23.3.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
import ast
import torch
import random
import os
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from deepctr_torch.inputs import SparseFeat, DenseFeat, VarLenSparseFeat
from deepctr_torch.models import DeepFM

# ==========================================
# ⚙️ 全局配置中心 (Global Config)
# ==========================================
class SteamConfig:
    # 1. 数据路径
    CSV_PATH = '../data/steam/deepfm_train_100k.csv' # 你的训练文件路径
    MODEL_PATH = 'deepfm_steam_weights.pth'          # 模型保存路径
    
    # 2. 特征参数
    MAX_TAG_LEN = 5          # 每个游戏最多取多少个 Tag
    EMBEDDING_DIM = 64       # 词向量维度 (建议 32 或 64)
    
    # 3. 网络结构
    # 建议不要太大，推荐 (256, 128) 或 (128, 128)
    DNN_HIDDEN_UNITS = (128, 64) 
    DNN_DROPOUT = 0.5
    
    # 4. 训练参数
    BATCH_SIZE = 1024
    EPOCHS = 20              # 20轮一般就够了
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    SEED = 2024

# 实例化配置
cfg = SteamConfig()
print(f"🔥 当前配置 -> 设备: {cfg.DEVICE} | 网络结构: {cfg.DNN_HIDDEN_UNITS}")

# --- 固定随机种子 (保证结果可复现) ---
def seed_everything(seed=2024):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(cfg.SEED)

# --- 序列补齐函数 ---
def pad_sequences(sequences, maxlen, value=0):
    """把变长的 Tag 列表补齐为定长"""
    result = np.full((len(sequences), maxlen), value, dtype=np.int32)
    for i, seq in enumerate(sequences):
        if len(seq) > 0:
            trunc = seq[:maxlen]
            result[i, :len(trunc)] = trunc
    return result

## Steam 数据加载与预处理

根据 Config 自动调整词表大小和特征维度。

核心逻辑：

1. 处理 tags_list: 它是字符串格式的列表 "[1663, 1774]"，需要还原并补齐长度。

2. 处理 price: 归一化。

3. 处理 item_id: 建立索引映射。

In [ ]:
def load_steam_data(csv_path, config):
    print(f"📂 正在加载数据: {csv_path} ...")
    # 1. 读取 CSV
    data = pd.read_csv(csv_path)
    
    # 2. 处理 Tag 列表字符串 "[1, 2]" -> [1, 2]
    # 你的数据里 tags_list 是字符串，必须转换
    data['tags_list'] = data['tags_list'].apply(lambda x: ast.literal_eval(x))
    
    # --- Tag 编码 ---
    all_tags = [tag for sublist in data['tags_list'] for tag in sublist]
    # 获取最大 Tag ID (假设 Tag ID 是数字且从1开始，如果不是连续的，建议用 LabelEncoder)
    # 这里为了保险，我们用 LabelEncoder 重新编一遍，保证 ID 连续
    tag_lbe = LabelEncoder()
    tag_lbe.fit(all_tags)
    max_tag_id = len(tag_lbe.classes_) + 1
    
    # 将原始 Tag 映射为 1~N 的索引 (0留给Padding)
    data['tags_list'] = data['tags_list'].apply(
        lambda x: [i + 1 for i in tag_lbe.transform(x)] if len(x) > 0 else []
    )
    
    # --- ItemID 编码 ---
    item_lbe = LabelEncoder()
    data['item_id_idx'] = item_lbe.fit_transform(data['item_id'])
    max_item_id = data['item_id_idx'].max() + 1
    
    # --- Price 归一化 ---
    mms = MinMaxScaler(feature_range=(0, 1))
    data['price_norm'] = mms.fit_transform(data[['price']])
    
    # 3. 序列补齐
    tags_padded = pad_sequences(list(data['tags_list']), maxlen=config.MAX_TAG_LEN, value=0)
    
    # 4. 定义特征列 (Feature Columns)
    fixlen_feature_columns = [
        SparseFeat('item_id_idx', vocabulary_size=max_item_id, embedding_dim=config.EMBEDDING_DIM),
        DenseFeat('price_norm', dimension=1)
    ]
    
    varlen_feature_columns = [
        VarLenSparseFeat(
            SparseFeat('tags', vocabulary_size=max_tag_id, embedding_dim=config.EMBEDDING_DIM),
            maxlen=config.MAX_TAG_LEN, combiner='mean', length_name=None
        )
    ]
    
    linear_feature_columns = fixlen_feature_columns + varlen_feature_columns
    dnn_feature_columns = fixlen_feature_columns + varlen_feature_columns
    
    # 5. 组装 Input
    model_input = {
        'item_id_idx': data['item_id_idx'].values,
        'price_norm': data['price_norm'].values,
        'tags': tags_padded
    }
    
    target = data['label'].values
    
    # 返回数据和从数据中统计出的特征定义
    return model_input, linear_feature_columns, dnn_feature_columns, target, data

# 测试一下
# input_dict, linear_cols, dnn_cols, target, df_raw = load_steam_data(cfg.CSV_PATH, cfg)

2025-12-15 21:52:28.847640: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-15 21:52:28.896481: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-15 21:52:30.200857: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Steam 模型训练 (Training)

In [ ]:
import matplotlib.pyplot as plt
from deepctr_torch.callbacks import EarlyStopping

class SteamConfig:
    CSV_PATH = '../data/steam/deepfm_train_100k.csv'
    MODEL_PATH = 'deepfm_steam_weights.pth'
    
    # --- 网络结构设计 (针对 1.5k item / 100k data 的黄金比例) ---
    MAX_TAG_LEN = 5
    EMBEDDING_DIM = 32       # 1500个物品，32维足够表达特征，太大容易过拟合
    DNN_HIDDEN_UNITS = (128, 64) # 两层足够捕捉高阶特征，不用太深
    DNN_DROPOUT = 0.5        # 数据量少，Dropout 开大点(0.5)防止死记硬背
    
    # --- 训练参数 ---
    BATCH_SIZE = 256         # 稍微调小 Batch Size，有助于模型跳出局部最优解，学得更细
    EPOCHS = 50              # 设置大一点，靠 EarlyStopping 来自动停
    LEARNING_RATE = 0.001    # Adam 默认值，通常不用改
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

cfg = SteamConfig()

In [ ]:
# 1. 加载数据 (复用之前的 load_steam_data 函数)
input_dict, linear_cols, dnn_cols, target, _ = load_steam_data(cfg.CSV_PATH, cfg)

# 2. 初始化模型
model = DeepFM(linear_feature_columns=linear_cols, 
               dnn_feature_columns=dnn_cols, 
               task='binary', 
               dnn_hidden_units=cfg.DNN_HIDDEN_UNITS,
               dnn_dropout=cfg.DNN_DROPOUT,
               device=cfg.DEVICE)

# 3. 编译 (注意 metrics)
model.compile("adam", "binary_crossentropy", metrics=["binary_crossentropy", "auc"])

# 4. 定义早停策略 (监控 val_loss，如果 3 轮都不下降了，就停止)
# 注意：DeepCTR-Torch 的 EarlyStopping 需要手动调用或自己写回# 1. 加载数据 (复用之前的 load_steam_data 函数)
input_dict, linear_cols, dnn_cols, target, _ = load_steam_data(cfg.CSV_PATH, cfg)

# 2. 初始化模型
model = DeepFM(linear_feature_columns=linear_cols, 
               dnn_feature_columns=dnn_cols, 
               task='binary', 
               dnn_hidden_units=cfg.DNN_HIDDEN_UNITS,
               dnn_dropout=cfg.DNN_DROPOUT,
               device=cfg.DEVICE)

# 3. 编译 (注意 metrics)
model.compile("adam", "binary_crossentropy", metrics=["binary_crossentropy", "auc"])

# 4. 定义早停策略 (监控 val_loss，如果 3 轮都不下降了，就停止)
# 注意：DeepCTR-Torch 的 EarlyStopping 需要手动调用或自己写回调，
# 这里 deepctr_torch 的 fit 并不直接支持 Keras 风格的 callbacks 列表，
# 但它返回的 history 足够我们要画图了。我们手动根据 epoch 设定简单的早停通常也可以，
# 或者直接跑完看曲线。这里为了简单，我们先跑完看曲线。

print(f"🚀 开始训练... (Items: ~1500, Data: {len(target)})")

# 5. 训练并获取历史记录
history = model.fit(input_dict, target, 
                    batch_size=cfg.BATCH_SIZE, 
                    epochs=cfg.EPOCHS, 
                    verbose=2, 
                    validation_split=0.2) # 划 20% 做验证集

# 6. 保存模型
torch.save(model.state_dict(), cfg.MODEL_PATH)
print("✅ 模型保存完毕。")

# ==========================================
# 📈 关键步骤：绘制 Loss 曲线
# ==========================================
def plot_learning_curves(history):
    # 提取数据
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    
    plt.figure(figsize=(12, 5))
    
    # 绘制 Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, 'b-', label='Training Loss')
    plt.plot(epochs, val_loss, 'r--', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    # 如果有 AUC 指标也可以画
    if 'auc' in history.history:
        auc = history.history['auc']
        val_auc = history.history['val_auc']
        plt.subplot(1, 2, 2)
        plt.plot(epochs, auc, 'b-', label='Training AUC')
        plt.plot(epochs, val_auc, 'r--', label='Validation AUC')
        plt.title('Training and Validation AUC')
        plt.xlabel('Epochs')
        plt.ylabel('AUC')
        plt.legend()
        plt.grid(True)
        
    plt.tight_layout()
    plt.show()

print("\n📊 正在生成训练曲线...")
plot_learning_curves(history)调，
# 这里 deepctr_torch 的 fit 并不直接支持 Keras 风格的 callbacks 列表，
# 但它返回的 history 足够我们要画图了。我们手动根据 epoch 设定简单的早停通常也可以，
# 或者直接跑完看曲线。这里为了简单，我们先跑完看曲线。

print(f"🚀 开始训练... (Items: ~1500, Data: {len(target)})")

# 5. 训练并获取历史记录
history = model.fit(input_dict, target, 
                    batch_size=cfg.BATCH_SIZE, 
                    epochs=cfg.EPOCHS, 
                    verbose=2, 
                    validation_split=0.2) # 划 20% 做验证集

# 6. 保存模型
torch.save(model.state_dict(), cfg.MODEL_PATH)
print("✅ 模型保存完毕。")

# ==========================================
# 📈 关键步骤：绘制 Loss 曲线
# ==========================================
def plot_learning_curves(history):
    # 提取数据
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    
    plt.figure(figsize=(12, 5))
    
    # 绘制 Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, 'b-', label='Training Loss')
    plt.plot(epochs, val_loss, 'r--', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    # 如果有 AUC 指标也可以画
    if 'auc' in history.history:
        auc = history.history['auc']
        val_auc = history.history['val_auc']
        plt.subplot(1, 2, 2)
        plt.plot(epochs, auc, 'b-', label='Training AUC')
        plt.plot(epochs, val_auc, 'r--', label='Validation AUC')
        plt.title('Training and Validation AUC')
        plt.xlabel('Epochs')
        plt.ylabel('AUC')
        plt.legend()
        plt.grid(True)
        
    plt.tight_layout()
    plt.show()

print("\n📊 正在生成训练曲线...")
plot_learning_curves(history)

Loading Steam data from: ../data/steam/train_data_Hardcore_Gamer.csv ...
Compressed Max Tag ID (Vocabulary Size): 296
Detected Max Tag ID: 296
🚀 使用设备: cuda
cuda
Train on 400 samples, validate on 100 samples, 7 steps per epoch
Epoch 1/100
0s - loss:  0.6793
Epoch 2/100
0s - loss:  0.6288
Epoch 3/100
0s - loss:  0.5850
Epoch 4/100
0s - loss:  0.5360
Epoch 5/100
0s - loss:  0.4781
Epoch 6/100
0s - loss:  0.4307
Epoch 7/100
0s - loss:  0.3929
Epoch 8/100
0s - loss:  0.3657
Epoch 9/100
0s - loss:  0.3397
Epoch 10/100
0s - loss:  0.3081
Epoch 11/100
0s - loss:  0.2746
Epoch 12/100
0s - loss:  0.2393
Epoch 13/100
0s - loss:  0.2073
Epoch 14/100
0s - loss:  0.1795
Epoch 15/100
0s - loss:  0.1579
Epoch 16/100
0s - loss:  0.1413
Epoch 17/100
0s - loss:  0.1303
Epoch 18/100
0s - loss:  0.1219
Epoch 19/100
0s - loss:  0.1152
Epoch 20/100
0s - loss:  0.1084
Epoch 21/100
0s - loss:  0.1026
Epoch 22/100
0s - loss:  0.0964
Epoch 23/100
0s - loss:  0.0903
Epoch 24/100
0s - loss:  0.0838
Epoch 25/100
0s

## 加载权重并启动 API 服务 (Loading & Serving)

In [ ]:
import torch
import pandas as pd
import threading
from flask import Flask, request, jsonify
from deepctr_torch.models import DeepFM

# 1. 准备环境
print("🔄 正在初始化服务，重新构建模型结构...")

# 为了获取 Feature Columns 结构，我们需要再调用一次数据加载函数
# (注意：这里我们复用之前的 load_steam_data，虽然效率略低但能保证结构绝对一致)
_, linear_cols, dnn_cols, _, df_raw = load_steam_data(cfg.CSV_PATH, cfg)

# 2. 初始化空模型 (关键：使用 cfg.DNN_HIDDEN_UNITS)
service_model = DeepFM(linear_feature_columns=linear_cols, 
                       dnn_feature_columns=dnn_cols, 
                       task='binary', 
                       dnn_hidden_units=cfg.DNN_HIDDEN_UNITS, # ✅ 确保与训练一致
                       dnn_dropout=cfg.DNN_DROPOUT,
                       device=cfg.DEVICE)

# 3. 加载权重
try:
    service_model.load_state_dict(torch.load(cfg.MODEL_PATH, map_location=cfg.DEVICE))
    service_model.eval() # 切换到评估模式
    print("✅ 模型权重加载成功！Ready to serve.")
except FileNotFoundError:
    print(f"❌ 未找到权重文件 {cfg.MODEL_PATH}，请先运行 Cell 4 进行训练。")

# ================= 搭建 Flask API (Notebook 版) =================
app = Flask(__name__)

@app.route('/recommend', methods=['POST'])
def recommend_api():
    try:
        req = request.json
        top_k = req.get('top_k', 3)
        
        # --- 简单全量检索逻辑 ---
        # 1. 构造 Input (直接使用内存中的 df_raw 处理结果)
        # 注意：实际生产中应把 pad_sequences 封装好给单条数据用，这里为了演示方便直接批量预测
        tags_padded = pad_sequences(list(df_raw['tags_list']), maxlen=cfg.MAX_TAG_LEN, value=0)
        
        input_dict = {
            'item_id_idx': df_raw['item_id_idx'].values,
            'price_norm': df_raw['price_norm'].values,
            'tags': tags_padded
        }
        
        # 2. 预测
        with torch.no_grad():
            pred_scores = service_model.predict(input_dict, batch_size=4096)
            
        # 3. 排序
        temp_df = df_raw.copy()
        temp_df['score'] = pred_scores
        top_items = temp_df.sort_values(by='score', ascending=False).head(top_k)
        
        # 4. 格式化
        results = []
        for _, row in top_items.iterrows():
            results.append({
                "id": str(row['item_id']),
                "title": row['title'],
                "score": float(row['score']),
                "tags": row.get('tag_names', 'Unknown')
            })
            
        return jsonify({"code": 200, "data": results})
        
    except Exception as e:
        return jsonify({"code": 500, "error": str(e)})

# ================= 启动后台线程 =================
def run_flask():
    # 端口设为 5000
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# 避免重复启动
if 'flask_thread' not in locals() or not flask_thread.is_alive():
    flask_thread = threading.Thread(target=run_flask)
    flask_thread.daemon = True
    flask_thread.start()
    print("🚀 服务已在后台启动！监听端口 5000...")
else:
    print("⚠️ 服务已经在运行中，无需再次启动。")

🔄 正在初始化服务，重新构建模型结构...
Loading Steam data from: ../data/steam/train_data_Hardcore_Gamer.csv ...
Compressed Max Tag ID (Vocabulary Size): 296
Detected Max Tag ID: 296
✅ Steam 模型加载成功！
🚀 服务已启动！监听端口 5000...
你可以运行下一个 Cell 来测试接口。
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.250.145.130:5000
Press CTRL+C to quit


## 测试服务 (Client Test)

In [ ]:
import requests
import time

# 等待服务就绪
time.sleep(1)

url = "http://127.0.0.1:5000/recommend"

print(f"📡 发送请求到: {url} ...")
payload = {
    "type": "steam", 
    "top_k": 3
}

try:
    resp = requests.post(url, json=payload, timeout=5)
    if resp.status_code == 200:
        print("\n✅ 推荐成功！结果如下：")
        data = resp.json().get('data', [])
        for item in data:
            print(f"  - [{item['score']:.4f}] {item['title']} ({item['tags']})")
    else:
        print(f"❌ 请求失败: {resp.text}")
except Exception as e:
    print(f"❌ 连接错误: {e}")

127.0.0.1 - - [15/Dec/2025 21:57:05] "POST /recommend HTTP/1.1" 200 -


📡 请求 Steam 推荐...
Steam Response: {'code': 200, 'data': [{'cover': 'https://shared.fastly.steamstatic.com/store_item_assets/steam/apps/3606480/bbdbbfce8a336864a974ee49419dabb6ccce5793/capsule_231x87_schinese.jpg?t=1765589367', 'id': '3606480', 'score': 0.9585115313529968, 'tags': '动作,射击,肉鸽', 'title': 'Call of Duty®: Black Ops 7'}, {'cover': 'https://shared.fastly.steamstatic.com/store_item_assets/steam/apps/1466860/capsule_231x87.jpg?t=1762295217', 'id': '1466860', 'score': 0.9539361000061035, 'tags': '多人', 'title': 'Age of Empires IV: Anniversary Edition'}], 'type': 'steam'}
------------------------------
